In [1]:
import search_engine.util as util
import re
import os
import pandas
import re
import sys
import copy

In [2]:
WIKI_DUMP_FILENAME = 'data/frwiki-20151226-pages-articles.xml'
WIKI_DIR = 'data/wiki'
WIKI_N_LINES = 246871655

In [3]:
if not os.path.exists(WIKI_DIR):
    os.mkdir(WIKI_DIR)

Functions to print progress during file browsing

In [4]:
def pretty_number(n):
    a = str(n)[::-1]
    return (' '.join([a[i:i+3] for i in range(0, len(a), 3)]))[::-1]

def print_progress(current_line):
    global last_line, last_print, WIKI_N_LINES
    if current_line - last_line < 100000:
        return
    sys.stdout.write('\b' * len(last_print))
    sys.stdout.write('\r')
    s = '{} / {}'.format(pretty_number(current_line), pretty_number(WIKI_N_LINES))
    sys.stdout.write(s)
    sys.stdout.flush()
    last_line = current_line
    last_print = s

In [5]:
page_to_id = {} # page title => [ids]
id_to_page = {} # id => page title

## Get title and attach an id of each page

In [6]:
fd_wiki_dump = open(WIKI_DUMP_FILENAME, 'r')

in_page = False
page_title = None
page_id = None

re_id = '.*<id>(\d+)</id>.*'
re_title = '.*<title>(.+)</title>.*'

last_print = ''
last_line = 0
line_count = 0

page_id = 0

for line in fd_wiki_dump:
    line = line[:-1]
    line = line.strip()
    line = util.process_word(line.decode('utf-8'))
    if in_page:
        if '</page' in line:
            in_page = False
            if page_title in page_to_id:
                page_to_id[page_title].append(page_id)
            else:
                page_to_id[page_title] = [page_id]
            id_to_page[page_id] = page_title
            page_id += 1
            page_title = None
        elif '<title' in line:
            match = re.match(re_title, line)
            if match is None:
                continue
            page_title = match.group(1)
            page_title = util.normalize_page_content(page_title)
    else:
        if '<page' in line:
            in_page = True
    print_progress(line_count)
    line_count += 1

fd_wiki_dump.close()

246 800 000 / 246 871 655

## Sort both page_to_id and id_to_page

In [7]:
for k,v in page_to_id.items():
    page_to_id[k] = sorted(v)

page_to_id = sorted(page_to_id.items())
id_to_page = sorted(id_to_page.items())

## Save page_to_id in csv file

In [8]:
with open(os.path.join(WIKI_DIR, 'page_to_id.csv'), 'w') as fd:
    fd.write('page@ids\n')
    for page,list_id in page_to_id:
        fd.write('{}@{}\n'.format(page, ' '.join([str(k) for k in list_id])))
    

## Save id_to_page in csv file

In [9]:
with open(os.path.join(WIKI_DIR, 'id_to_page.csv'), 'w') as fd:
    fd.write('id@page\n')
    for page_id,page in id_to_page:
        fd.write('{}@{}\n'.format(page_id, page))
    